In [1]:

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
import time
import pandas as pd
import random 
import csv 
from pathlib import Path




In [2]:
option = webdriver.FirefoxOptions()
option.set_preference("browser.download.manager.showWhenStarting", False)
option.set_preference('browser.helperApps.neverAsk.saveToDisk',
                       'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                       ';application/vnd.ms-excel'
                       ';application/xls'
                       ';application/xlsx')
option.set_preference("permissions.default.stylesheet", 2)
option.set_preference("permissions.default.image", 2)
option.set_preference("javascript.enabled", False)


In [3]:
# Open browser and hit the link

driver= webdriver.Firefox(options=option)

driver.get('https://mnregaweb4.nic.in/netnrega/state_html/empstatusnewall_scst.aspx?page=S&lflag=eng&state_name=UTTAR%20PRADESH&state_code=31&fin_year=2021-2022&source=national&Digest=zSgQ2v/Smh8sUQFJPau9Rg')

assert "NREGA" in driver.title


In [4]:
#install lxml to parse entire table into dataframe format 
# !pip3 install lxml 


In [5]:

##total districts 
districts = driver.find_elements('xpath', '//*[@id="form1"]/div[3]/center/table/tbody/tr[position()>4]/td[2]/a')

District  = [a.text for a in districts ]

#print(District)

District_urls = [a.get_attribute('href') for a in districts]

##all district urls 

# print(District_urls)



In [6]:

final_dfs = [] 

##Now find all the block links for each district 

## for each district url 
for i in range(0, len(District_urls), 1): 

    #click on the district url 
    
    driver.get(District_urls[i])
    
    time.sleep(1)

    ##find all the block links and text and save them for later use 
    b_urls = driver.find_elements('xpath', '/html/body/form/div[3]/center/table/tbody/tr[position()>4]/td[2]/a')

    #block name 
    block = [a.text for a in b_urls]

    #block url 
    block_urls = [a.get_attribute("href") for a in b_urls]


    #local variable to append all the tables into one dataframe    
    k =0 

    dfs_list=[]
    #for district i all block urls are saved in block_urls list 
    #click on each block url and save the data 
    for url in block_urls:

        # this will print the url of the block which is going to be scraped in district i 
        urls = "'"+url+"'"
        #print(urls,'\n')

        #this should open the each block url of district i 
        driver.get(url)

        tables = WebDriverWait(driver, 20).until(ec.presence_of_all_elements_located((By.TAG_NAME, "table")))

        #print(len(tables))

        for table in tables:
           lst= pd.read_html(table.get_attribute('outerHTML'))
        
        ##now get the table from a block in all the blocks under a district and save it in the the dataframe 
        df = pd.DataFrame(lst[0])

        #add district name
        df['District'] = District[i]

        #add block name for district i 

        df['Block'] = block[k]

        ##update the local varriable to update the block 

        k+= 1

        ##merge the dataframe into the previous one 
        dfs_list.append(df)
     
        ##This should print all the blocks' tables of a district 
        #print(dfs_list) 

        time.sleep(3)
        driver.back()
    
    ##save the data for one district 
    final_df = pd.concat(dfs_list, axis=0) 

    #append the district level data into the previous one 

    final_dfs.append(final_df)

    ##go back to previous district 
    driver.back()

#save the entire district data 
global Final_dataset 

Final_Dataset = pd.concat(final_dfs, axis=0)

Final_Dataset.to_csv("trial_Narega.csv")


